In [1]:
from datasets import load_dataset
import datasets

In [ ]:
features = datasets.Features({'id': datasets.Value(dtype='string'),
                              })

In [8]:
ds = load_dataset('json', data_files="../../.convokit/saved-corpora/supreme-corpus/utterances.jsonl")

In [ ]:
ds

In [ ]:
ds["train"][0]["meta"]

In [ ]:
ds["train"][1]["meta"]

In [ ]:
ds["train"][1]['text'].split("\n")

In [2]:
from datasets import load_dataset

dataset = load_dataset("audiofolder", data_dir="./data/audios/")

Resolving data files:   0%|          | 0/2317 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/328 [00:00<?, ?it/s]

In [ ]:
dataset.push_to_hub("labicquette/audio_argument_raws")

In [ ]:
print(dataset["validation"][0]['audio']["array"])

In [ ]:
dataset["validation"][0]

In [3]:
from datasets import load_dataset

ds = load_dataset("./source/load_SCOTUS.py",trust_remote_code=True)


In [ ]:
ds["validation"]

In [4]:
trans = load_dataset('json', data_files="../../.convokit/saved-corpora/supreme-corpus/utterances.jsonl")

In [ ]:
trans["train"][0]["meta"]["case_id"]

In [ ]:
from tqdm.notebook import tqdm
ids = []
for ex in tqdm(trans["train"]):
    ids += [ex["meta"]["case_id"]]

In [5]:
from tqdm.notebook import tqdm
ids = []
for ex in tqdm(trans["train"]):
    ids += [ex["meta"]["case_id"]]

import numpy as np 
un_ids = np.unique(ids)

scotus_ids = []
#for split in ds: 
for row in tqdm(ds["validation"]): 
    scotus_ids += [row["year_argued"] + "_" +row["docket_number"]]

  0%|          | 0/1700789 [00:00<?, ?it/s]

  0%|          | 0/369 [00:00<?, ?it/s]

In [6]:
ids_arr = np.array(ids)

In [7]:
scotus_uniques = np.unique(scotus_ids)

In [8]:
small_transcripts = trans["train"].select(list(np.where(np.isin(ids_arr, scotus_uniques))[0]))

In [ ]:
len(small_transcripts)

In [ ]:
small_transcripts[0]

In [9]:
small_transcripts = small_transcripts.add_column("audios", [{"audios":[{"path": [], "array":None,'sampling_rate':0}]}] * len(small_transcripts))
"""
{'audio': {'path': '/home/labicquette/M2/projet/data/audios/validation/14 Penn Plaza LLC v. Pyett-556-247.mp3',
           'array': array([0.00000000e+00, 8.96333246e-14, 3.08656312e-13, ...,
         5.10093400e-07, 5.49440131e-07, 6.17488581e-07]),
           'sampling_rate': 22050},
 'label': 1}
"""

"\n{'audio': {'path': '/home/labicquette/M2/projet/data/audios/validation/14 Penn Plaza LLC v. Pyett-556-247.mp3',\n           'array': array([0.00000000e+00, 8.96333246e-14, 3.08656312e-13, ...,\n         5.10093400e-07, 5.49440131e-07, 6.17488581e-07]),\n           'sampling_rate': 22050},\n 'label': 1}\n"

In [ ]:
small_transcripts["audios"]

In [10]:
from source.utils import get_file
def get_path(case_id, data_tuple, dataset):
    ty, td, vy, vd = data_tuple
    ty = np.array(ty)
    td = np.array(td)
    vy = np.array(vy)
    vd = np.array(vd)
    case_id = case_id.split("_")
    cy = case_id[0]
    cd = case_id[1]
    train_idx = np.intersect1d(np.where(ty == cy),np.where(td == cd))
    #print(train_idx)
    validation_idx = np.intersect1d(np.where(vy == cy),np.where(vd == cd))
    #print(validation_idx)
    if len(validation_idx) == 0:
        #print(dataset["train"][int(train_idx[0])])
        print("problem")
        path = "./data/audios/train/" + get_file(dataset["train"][int(train_idx[0])], "SCOTUS")
        return 0
    else : 
        path = "./data/audios/validation/" + get_file(dataset["validation"][int(validation_idx[0])], "SCOTUS")
    return path

In [ ]:
#from pydub import AudioSegment
import ffmpeg
(
ffmpeg.input("./data/audios/train/Cole v. Young-351-536.mp3", ss=0, to=30)
        .output("./test.mp3", codec='copy')
        .run(overwrite_output=True) 
)

In [11]:
import os 
import ffmpeg
def batch_audio_split(ex, idx, ds_tuple):
    case_id = ex["meta"]["case_id"]
    temp_file = get_path(case_id, ds_tuple, ds)
    if type(temp_file) == int:
        return
    end_file = temp_file.split("audios")[-1]
    case_dir = end_file.split("/")[-1]
    try:
        os.mkdir("./data/linked_audios"+end_file)
    except: 
        pass
    for start_t, stop_t in zip(ex["meta"]["start_times"], ex["meta"]["stop_times"]):
            (
            ffmpeg.input(temp_file+".mp3", ss=start_t, to=stop_t)#TODO EXTRACT TIMES 
                .output("./data/linked_audios"+end_file+"/"+case_dir+"__part-"+str(idx)+".mp3", codec='copy',loglevel="quiet")
                .run(overwrite_output=True) 
            )

Input #0, mp3, from './data/audios/validation/United States v. Mississippi Chemical Corporation-405-298.mp3':
  Metadata:
    title           : United States v. Mississippi Chemical Corporation - Oral Argument - January 10, 1972
    artist          : Supreme Court of the United States
    album           : 1971 Term
    genre           : Speech
    composer        : Oyez
    album_artist    : Oyez
    publisher       : Oyez
    copyright       : 2015 Oyez, Inc. Licensed to the public under http://creativecommons.org/licenses/by-nc/4.0/ verify at https://api.oyez.org/cases/1971/70-52
  Duration: 00:59:28.54, start: 0.000000, bitrate: 32 kb/s
  Stream #0:0: Audio: mp3, 22050 Hz, mono, fltp, 32 kb/s
  Stream #0:1: Video: png, rgb24(pc), 1250x1250 [SAR 11811:11811 DAR 1:1], 90k tbr, 90k tbn, 90k tbc (attached pic)
    Metadata:
      comment         : Other
Output #0, mp3, to './data/linked_audios/validation/United States v. Mississippi Chemical Corporation-405-298/United States v. Mississ

In [16]:
ds_tuple = (ds["train"]["year_argued"],ds["train"]["docket_number"],ds["validation"]["year_argued"],ds["validation"]["docket_number"])
small_transcripts.map(lambda example,idx : batch_audio_split(example, idx, ds_tuple), with_indices=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/81847 [00:00<?, ? examples/s]

Exception in thread Thread-20 (_handle_results):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/labicquette/.local/lib/python3.10/site-packages/multiprocess/pool.py", line 579, in _handle_results
    task = get()
  File "/home/labicquette/.local/lib/python3.10/site-packages/multiprocess/connection.py", line 254, in recv
    return _ForkingPickler.loads(buf.getbuffer())
  File "/home/labicquette/.local/lib/python3.10/site-packages/dill/_dill.py", line 303, in loads
    return load(file, ignore, **kwds)
  File "/home/labicquette/.local/lib/python3.10/site-packages/dill/_dill.py", line 289, in load
    return Unpickler(file, ignore=ignore, **kwds).load()
  File "/home/labicquette/.local/lib/python3.10/site-packages/dill/_dill.py", line 444, in load
    obj = StockUnpickler.load(self)
TypeError: 

AssertionError: Cannot have cache with result_hander not alive

In [ ]:
import os 
ds_tuple = (ds["train"]["year_argued"],ds["train"]["docket_number"],ds["validation"]["year_argued"],ds["validation"]["docket_number"])
temp_file = ""
temp_case_id = ""
for ex in tqdm(small_transcripts.select(list(range(50)))):
    print(ex)
    case_id = ex["meta"]["case_id"]
    start = ex["meta"]["start_times"]
    end = ex["meta"]["stop_times"]
    if case_id == temp_case_id:
        print("ok")
        for start_t, stop_t in zip(ex["meta"]["start_times"], ex["meta"]["stop_times"]):
            cpt += 1
            (
            ffmpeg.input(temp_file+".mp3", ss=start_t, to=stop_t)#TODO EXTRACT TIMES 
                .output("./data/linked_audios"+end_file+"/"+case_dir+"__part-"+str(cpt)+".mp3", codec='copy')
                .run(overwrite_output=True) 
            )
    else:
        cpt = 0
        temp_case_id = ex["meta"]["case_id"]
        temp_file = get_path(case_id, ds_tuple, ds)
        print(case_id, temp_file)
        print("TEMP_FILE",temp_file)
        end_file = temp_file.split("audios")[-1]
        print(end_file)
        case_dir = end_file.split("/")[-1]
        print("./data/linked_audios"+end_file+"/"+case_dir+"__part-"+str(cpt)+".mp3")
        os.makedirs("./data/linked_audios"+end_file)
        for start_t, stop_t in zip(ex["meta"]["start_times"], ex["meta"]["stop_times"]):
            cpt += 1
            (
            ffmpeg.input(temp_file+".mp3", ss=start_t, to=stop_t)#TODO EXTRACT TIMES 
                .output("./data/linked_audios"+end_file+"/"+case_dir+"__part-"+str(cpt)+".mp3", codec='copy')
                .run(overwrite_output=True) 
            )
        
